In [1]:
from gurobipy import *
import pandas as pd
import numpy as np

In [2]:
df_GSW = pd.read_csv("GSW_player_sl.csv")
df_GSW['Time_limit_in_minutes'] = df_GSW['Time_limit']*1.5
df_GSW.iloc[1,5]=48
df_GSW.iloc[13,0]=0
df_GSW.iloc[12,0]=0
df_GSW.iloc[6,0]=1
df_GSW.iloc[9,0]=1
print(df_GSW[['selected','Player','Pos','Fantasy_point','Time_limit_in_minutes']])

    selected                 Player Pos  Fantasy_point  Time_limit_in_minutes
0          1         Andre Iguodala   G           3.67                 30.795
1          1         Andrew Wiggins   F           2.48                 48.000
2          1          Chris Chiozza   G           2.04                 29.250
3          1             Damion Lee   G           2.01                 36.000
4          1         Draymond Green   F           4.14                 45.195
5          1         Gary Payton II   G           2.52                 23.850
6          1            Jeff Dowtin   G          -0.59                 33.150
7          1       Jonathan Kuminga   C          -0.24                 29.850
8          1           Jordan Poole   G           2.38                 35.250
9          1  Juan Toscano-Anderson   F           0.19                 27.795
10         1           Kevon Looney   C           1.19                 25.350
11         0            Moses Moody   G          -0.12          

### Baseline Model 1

In [3]:
# model set-up

def model_setup():
    
    m = Model("Baseline Model 1")
    x = m.addVars(T,vtype = GRB.BINARY, name = 'x')
    m.setObjective( quicksum(x[i,j,h] * F[j] for i in range(I) for j in range(J) for h in range(H)), GRB.MAXIMIZE)
    m.addConstrs( (x[i,j,h] <= z[j][h] for i in range(I) for j in range(J) for h in range(H)), "position")
    m.addConstrs( (quicksum(x[i,j,0] for j in range(J)) == 2 for i in range(I)), "guard")
    m.addConstrs( (quicksum(x[i,j,1] for j in range(J)) == 2 for i in range(I)), "forward")
    m.addConstrs( (quicksum(x[i,j,2] for j in range(J)) == 1 for i in range(I)), "center")
    m.addConstrs( (quicksum(x[i,j,h] for i in range(I) for h in range(H)) - quicksum(p[j][h] for h in range(H)) <= gap for j in range(J)), "physical limitation")
    m.setParam('OutputFlag', False)
    return m

In [4]:
df = df_GSW.iloc[:,:]
df = df[df["selected"] == 1].reset_index(drop=True)

In [5]:
df

,selected,Player,Pos,Fantasy_point,Time_limit,Time_limit_in_minutes
0,1,Andre Iguodala,G,3.67,20.53,30.795
1,1,Andrew Wiggins,F,2.48,32.27,48.000
2,1,Chris Chiozza,G,2.04,19.50,29.250
3,1,Damion Lee,G,2.01,24.00,36.000
4,1,Draymond Green,F,4.14,30.13,45.195
5,1,Gary Payton II,G,2.52,15.90,23.850
6,1,Jeff Dowtin,G,-0.59,22.10,33.150
7,1,Jonathan Kuminga,C,-0.24,19.90,29.850
8,1,Jordan Poole,G,2.38,23.50,35.250
9,1,Juan Toscano-Anderson,F,0.19,18.53,27.795


In [6]:
# parameters set-up

F = df["Fantasy_point"].to_list()
print("F:",F,"\n")
# F = [6.4,3.2,1.4,0.6,0,0.1,2.1,0.2,-0.4,0.3,2.1,-1.1]

z = []
p = []
for i in range(len(df)):
    if df.iloc[i,2] == 'G':
        z.append([1,0,0])
        p.append([min(df.iloc[i,4]*1.5,48),0,0])
    elif df.iloc[i,2] == 'F':
        z.append([0,1,0])
        p.append([0,min(df.iloc[i,4]*1.5,48),0])        
    else:
        z.append([0,0,1])
        p.append([0,0,min(df.iloc[i,4]*1.5,48)])
print("z:",z,"\n")
print("p:",p)

I = 48
J = 12
H = 3
gap = 0

temp = []
for i in range(I):
    for j in range(J):
        for h in range(H):
            temp.append((i,j,h))
T = tuplelist(temp)

F: [3.67, 2.48, 2.04, 2.01, 4.14, 2.52, -0.59, -0.24, 2.38, 0.19, 1.19, 4.3] 

z: [[1, 0, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0], [0, 0, 1], [1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0]] 

p: [[30.795, 0, 0], [0, 48, 0], [29.25, 0, 0], [36.0, 0, 0], [0, 45.195, 0], [23.85, 0, 0], [33.150000000000006, 0, 0], [0, 0, 29.849999999999998], [35.25, 0, 0], [0, 27.795, 0], [0, 0, 25.349999999999998], [47.849999999999994, 0, 0]]


In [23]:
# setup and solve the modle
m = model_setup()
m.optimize()

#  Print optimal solutions and optimal value
print("\n Optimal solution:")
i_th = 1
j_th = 1
h_th = 1
pos1 = "Guard:  \t"
pos2 = "Forward:\t"
pos3 = "Center:\t"
for var in m.getVars():
    if var.x > 0 and h_th == 1:
        pos1 = pos1 + "Player No." + str(j_th) + "\t"
    elif var.x > 0 and h_th == 2:
        pos2 = pos2 + "Player No." + str(j_th) + "\t"
    elif var.x > 0 and h_th == 3:
        pos3 = pos3 + "Player No." + str(j_th) + "\t"
    if j_th == 12 and h_th == 3:
        print("Minute",str(i_th),":\n")
        print("\t",pos1,"\n\t",pos2,"\n\t",pos3,"\n")
        pos1 = "Guard:  \t"
        pos2 = "Forward:\t"
        pos3 = "Center:\t"
        j_th = 0
        i_th = i_th + 1
    if h_th == 3:
        h_th = 0
        j_th = j_th + 1
    h_th = h_th + 1

print("\n Optimal Score:")
print('Obj:', m.objVal)


 Optimal solution:
Minute 1 :

	 Guard:  	Player No.6	Player No.12	 
	 Forward:	Player No.2	Player No.5	 
	 Center:	Player No.8	 

Minute 2 :

	 Guard:  	Player No.6	Player No.12	 
	 Forward:	Player No.2	Player No.5	 
	 Center:	Player No.8	 

Minute 3 :

	 Guard:  	Player No.1	Player No.12	 
	 Forward:	Player No.2	Player No.5	 
	 Center:	Player No.8	 

Minute 4 :

	 Guard:  	Player No.1	Player No.12	 
	 Forward:	Player No.2	Player No.5	 
	 Center:	Player No.8	 

Minute 5 :

	 Guard:  	Player No.6	Player No.12	 
	 Forward:	Player No.2	Player No.5	 
	 Center:	Player No.8	 

Minute 6 :

	 Guard:  	Player No.1	Player No.12	 
	 Forward:	Player No.2	Player No.5	 
	 Center:	Player No.8	 

Minute 7 :

	 Guard:  	Player No.6	Player No.12	 
	 Forward:	Player No.2	Player No.5	 
	 Center:	Player No.8	 

Minute 8 :

	 Guard:  	Player No.6	Player No.12	 
	 Forward:	Player No.2	Player No.5	 
	 Center:	Player No.11	 

Minute 9 :

	 Guard:  	Player No.1	Player No.12	 
	 Forward:	Player No.2	Player No.

In [24]:
# Relax the time constraints

print("\n Gap \t Optimal Score \t Compared with the previous model")
for i in range(10):
    gap = (i) * 3
    
    # setup and solve the model
    m1 = model_setup()   
    m1.optimize()

    #  print optimal solutions and compare them    
    print(" ",gap,"\t",m1.objVal,"\t",m1.objVal - m.objVal)



 Gap 	 Optimal Score 	 Compared with the previous model
  0 	 690.2199999999999 	 0.0
  3 	 711.5899999999998 	 21.36999999999989
  6 	 719.3299999999998 	 29.1099999999999
  9 	 727.0699999999997 	 36.849999999999795
  12 	 734.8099999999997 	 44.589999999999804
  15 	 742.5499999999997 	 52.329999999999814
  18 	 750.2899999999995 	 60.069999999999595
  21 	 754.5799999999996 	 64.35999999999967
  24 	 757.4399999999996 	 67.21999999999969
  27 	 757.4399999999996 	 67.21999999999969
